In [ ]:
from semantic_kernel import __version__

__version__

In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

In [ ]:
import pandas as pd
from azure.identity.aio import DefaultAzureCredential
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread
from blood_pressure_plugin import BloodPressurePlugin  # Tu plugin personalizado

In [ ]:


async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        # Subir archivo con información médica
        file: OpenAIFile = await client.agents.upload_file_and_poll(
            file_path="clinical_bp_guidelines.md", purpose="assistants"
        )

        # Crear vector store con el archivo
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="bp_vector_store"
        )

        # Instrucciones del agente especializado en presión arterial
        AGENT_NAME = "BloodPressureRAGAgent"
        AGENT_INSTRUCTIONS = """
        Eres un asistente experto en presión arterial. Sólo puedes responder preguntas usando información del documento cargado.

        - Si la pregunta no tiene soporte en el documento, responde: "Lo siento, el documento no contiene suficiente información para responder eso."
        - No uses conocimiento externo, no asumas ni generes respuestas hipotéticas.
        - Responde de forma concisa y clínica cuando sea relevante.
        """

        # Crear herramienta de búsqueda basada en archivo
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # Crear definición del agente
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )

        # ✅ Crear instancia del plugin personalizado
        plugin = BloodPressurePlugin()

        # Crear instancia del agente con el plugin incluido
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
            plugins=[plugin]  # Se puede agregar más de un plugin si lo deseas
        )

        thread: AzureAIAgentThread | None = None

        # Preguntas de prueba
        user_inputs = [
            "¿Cuál es el rango normal de presión arterial según el documento?",
            "¿Cuáles son los factores que afectan la presión sistólica?",
            "Predice presión arterial para 5 pacientes sintéticos."
            "Genera 20 registros de datos de presión arterial y guárdalos en un CSV llamado 'bp_sintetico.csv'",
            "Dame 5 ejemplos de pacientes con presión alta y nivel de estrés alto",
            "Genera datos para modelar la presión arterial de mujeres de 40 años",
            "Quiero simular registros clínicos de 100 personas"
        ]

        try:
            for user_input in user_inputs:
                print(f"\n# Usuario: {user_input}")
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\n🧹 Limpieza completada: agente, archivo, vector store y thread eliminados.")

# Ejecutar
await main()
